In [1]:
# Basic Tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import matplotlib.colors as mcolors
#import seaborn as sns
from scipy.stats import randint, uniform, norm, expon, lognorm
from scipy.spatial import distance_matrix

from astropy.table import Table#, vstack, hstack, join
#import astropy.units as u
#import corner
import os

In [2]:
os.chdir('/home/kwalsen/Documents/ERIS/networks')

# Notes

- Most stars in clusters have up to 25 element abundances measured, but in some cases there are clusters that have less, down to 21 measured element abundances. So in X[j] I would like, if useful, to have NaNs in the elements that dont have the respective measurement, and then for the statistics just ommit the NaN values.

- There are clusters with 1 star (may consider doing histogram of number of stars)

# Data

In [3]:
abusolar_cluster_giants = Table.read('data/abusolar_cluster_giants.csv')
abusolar_star_giants_individual_stars = Table.read('data/abusolar_star_giants_individual_stars.csv')

In [4]:
abusolar_cluster_giants = pd.read_csv('data/abusolar_cluster_giants.csv')
abusolar_star_giants_individual_stars = pd.read_csv('data/abusolar_star_giants_individual_stars.csv')

In [5]:
# This because I want to make a function to setup the variables before MCMC, using the Tables as input
cluster_df = abusolar_cluster_giants
clusters_stars_df = abusolar_star_giants_individual_stars

In [6]:
print(cluster_df.columns)
cluster_cols = ['Nstars','cluster','MeanSNR','ageNN','err_ageNN','age','err_age','dist','X','Y','Z','ageNN_old','age_old']
cluster_df[cluster_cols]

Index(['Nstars', 'cluster', 'MeanSNR', 'Na 1', 'errNa 1', 'NNa 1', 'Al 1',
       'errAl 1', 'NAl 1', 'Mg 1',
       ...
       'Ba2Fe1', 'errBa2Fe1', 'Ce2Fe1', 'errCe2Fe1', 'Pr2Fe1', 'errPr2Fe1',
       'Nd2Fe1', 'errNd2Fe1', 'ageNN_old', 'age_old'],
      dtype='object', length=136)


,Nstars,cluster,MeanSNR,ageNN,err_ageNN,age,err_age,dist,X,Y,Z,ageNN_old,age_old
0,4.0,UBC_3,116.696970,8.301030,0.062,0.200000,0.020635,1644.0,1172.0,1145.0,136.0,8.301030,0.200000
1,1.0,NGC_6475,302.000000,8.350000,0.062,0.223872,0.031960,283.0,282.0,-20.0,-22.0,8.350000,0.223872
2,12.0,NGC_6705,68.963087,8.480000,0.062,0.301995,0.043113,2164.0,1921.0,991.0,-104.0,8.480000,0.301995
3,5.0,NGC_3532,176.800000,8.600000,0.048,0.398107,0.044000,498.0,166.0,-469.0,12.0,8.600000,0.398107
4,1.0,Stock_1,211.000000,8.640000,0.048,0.436516,0.048246,410.0,203.0,356.0,14.0,8.640000,0.436516
5,5.0,UBC_215,71.800000,8.650000,0.048,0.446684,0.049369,1372.0,-1099.0,-814.0,-107.0,8.650000,0.446684
6,10.0,NGC_2099,69.350000,8.650000,0.048,0.446684,0.049369,1384.0,-1380.0,56.0,74.0,8.650000,0.446684
7,2.0,NGC_6281,135.000000,8.710000,0.048,0.512861,0.056684,532.0,520.0,-112.0,18.0,8.710000,0.512861
8,3.0,NGC_6645,74.333333,8.710000,0.048,0.512861,0.056684,1750.0,1680.0,474.0,-110.0,8.710000,0.512861
9,2.0,FSR_0850,67.191489,8.710000,0.048,0.512861,0.056684,2126.0,-2121.0,-130.0,-85.0,8.710000,0.512861


In [7]:
print(clusters_stars_df.columns)
star_cols = ['Unnamed: 0','cluster','star','element','synth_x_over_h','synth_x_over_h_err','synth_absolute_abund','synth_absolute_abund_err']
clusters_stars_df[star_cols]

Index(['Unnamed: 0', 'cluster', 'star', 'element', 'synth_teff',
       'synth_teff_err', 'synth_logg', 'synth_logg_err', 'synth_MH',
       'synth_MH_err', 'synth_vmic', 'synth_vmic_err', 'synth_vmac',
       'synth_vmac_err', 'synth_vsini', 'synth_vsini_err', 'synth_R',
       'synth_R_err', 'vel_atomic', 'vel_atomic_err', 'synth_x_over_h',
       'synth_x_over_h_err', 'synth_absolute_abund',
       'synth_absolute_abund_err', 'synth_nlines', 'snr'],
      dtype='object')


,Unnamed: 0,cluster,star,element,synth_x_over_h,synth_x_over_h_err,synth_absolute_abund,synth_absolute_abund_err
0,25,FSR_0278,G2180302768725634688,Al 1,0.059541,0.0142,6.5259,0.0127
1,26,FSR_0278,G2180302768725634688,Ba 2,0.076139,0.0163,2.1975,0.0064
2,27,FSR_0278,G2180302768725634688,Ca 1,0.029721,0.0327,6.3626,0.0866
3,28,FSR_0278,G2180302768725634688,Co 1,0.023984,0.0397,4.9239,0.1103
4,29,FSR_0278,G2180302768725634688,Cr 1,-0.010506,0.0941,5.6099,0.0788
...,...,...,...,...,...,...,...,...
5193,6235,UBC_6,G1989397554090593920,V 1,-0.115689,0.0181,3.7584,0.0641
5194,6236,UBC_6,G1989397554090593920,Y 2,0.048903,0.0050,2.2095,0.0411
5195,6237,UBC_6,G1989397554090593920,Zn 1,-0.057484,0.1162,4.2618,0.1144
5196,6238,UBC_6,G1989397554090593920,Zr 1,-0.156725,0.2482,2.5163,0.2468


# Setting T and X

In [8]:
clusters = np.array(cluster_df['cluster']) # Cluster names
K = len(clusters) # Number of clusters

# Setting T
T_m, T_sd = np.array(cluster_df['age']), np.array(cluster_df['err_age'])

# We reorder the arrays based on cluster Age T_m from oldest to youngest
cluster_sort_idx = np.argsort(T_m)[::-1]
clusters = clusters[cluster_sort_idx]
T_m, T_sd = T_m[cluster_sort_idx], T_sd[cluster_sort_idx]

# Setting X
# Approach in case I want to keep all the elements and set as NaN for the stars with missing ones.
X = [] # List with matrices X[k] k=1,...,K of shape N_cluster_stars x N_elements
elements = np.unique(clusters_stars_df['element']) # All the available element abundances (in order)
element_idx = lambda element: np.where(elements == element)[0][0] # Mapping for element to idx in ordered array
for k, cluster in enumerate(clusters):
    cluster_mask = clusters_stars_df['cluster'] == cluster
    cluster_individual_stars = clusters_stars_df[cluster_mask]
    cluster_stars = np.unique(cluster_individual_stars['star'])
    star_elements = np.unique(cluster_individual_stars['element'])
    X_k = np.full((len(cluster_stars), len(elements)), np.nan) # Empty matrix
    for i, star in enumerate(cluster_stars):
        star_mask = cluster_individual_stars['star'] == star
        individual_star = cluster_individual_stars[star_mask]
        elements_idx = list(individual_star['element'].apply(element_idx)) # Map the available elements to their respective ordered index
        X_k[i,elements_idx] = individual_star['synth_x_over_h'] # Is this column for the abundance in X?
    X.append(X_k)

In [9]:
'''
Add in case I want to exclude elements missing in at least one of the stars, for all the clusters
# obs: by doing this I lose 7 elements
exclude_mask = np.ones(len(elements), dtype='bool')
exclude_idx = []
for k in range(K):
    for i in range(len(X[k])):
        idx_to_exclude = np.argwhere(np.isnan(X[k][i]))
        if len(idx_to_exclude) > 0:
            idx_to_exclude = idx_to_exclude[0][0]
            exclude_idx.append(idx_to_exclude)
exclude_idx = np.unique(exclude_idx)
exclude_mask[exclude_idx] = 0
for k in range(K):
    X[k] = X[k][:,exclude_mask]
''';

In [10]:
'''
# Add in case I want to exclude stars with missing values in any of the elements
# obs: by doing this I lose 3 clusters, since all of its stars have a missing element. And also I leave a cluster with 1 star
exclude_mask = np.ones(len(elements), dtype='bool')
exclude_idx = []
for k in range(K):
    exclude_idx_k = []
    for i in range(len(X[k])):
        idx_to_exclude = np.argwhere(np.isnan(X[k][i]))
        if len(idx_to_exclude) > 0:
            #print(k,i)
            exclude_idx_k.append(i)
        else:
            continue # To only have either empty lists or lists with the indexes to exclude
            #exclude_idx_k.append(None) # To see the clusters after the potential removal
    exclude_idx.append(exclude_idx_k)

X_ = []
exclude_cluster = []
cluster_mask = np.ones(len(clusters), dtype='bool')
for k, cluster in enumerate(clusters):
    if len(X[k]) > len(exclude_idx[k]): # If The stars I remove don't delete the whole cluster
        exclude_mask = np.ones(len(X[k]), dtype='bool')
        exclude_mask[exclude_idx[k]] = 0
        X[k] = X[k][exclude_mask,:]
        X_.append(X[k])
    else: # I lose all the stars in the cluster, I remove it.
        print('Removed cluster {}: {}'.format(k,cluster))
        exclude_cluster.append(k)

# Change arrays after removing the clusters
X = X_
cluster_mask[exclude_cluster] = 0
clusters = clusters[cluster_mask]
K = len(clusters)
T_m, T_sd = T_m[cluster_mask], T_sd[cluster_mask]

# TO ADD: If necessary, Change the dfs removing rows involving the removed clusters and rearange with the respective arrays
''';

# Setting prior for T, G, A

In [11]:
np.random.seed(2808)

- Decide wether set the priors as functions or arrays

In [12]:
# If priors are set as functions (Improve if finally needed)
def Pr_T(loc, scale, size, **kwargs):
    return norm.rvs(loc, scale, size, **kwargs)

def Pr_G(s,loc,scale):
    return lognorm(s=s,loc=loc,scale=scale)

def Pr_A(loc,scale,size,**kwargs):
    return expon.rvs(loc,scale,size,**kwargs)

print('Pr_T')
print(Pr_T(loc=T_m, scale=T_sd,size=K))

print('Pr_G')
G_m, G_sd = 0, 1 # Don't know the actual values yet
s = 1 # Shape parameter for distribution, don't know actual useful value for this yet
N = K # Bins of equal probability, don't know if is K or a value sufficiently high
G_bins = randint.rvs(low=1, high=N+1, loc=0, size=K)
print(G_bins)
print(Pr_G(s,G_m,G_sd).ppf(q=(G_bins-0.5)/N))

print('Pr_A')
cluster_R = np.array(cluster_df['Rgc']).reshape(-1,1)[cluster_sort_idx] # Is Rgc the radius in pc? If not, where to find cluster radius
A_m = distance_matrix(cluster_R, cluster_R, p=1)
A_m = np.where(A_m!=0, 1/A_m,0) # Find a better way to compute it and avoid warning
print(A_m)
print(Pr_A(loc=0,scale=A_m, size=(K,K)))    

Pr_T
[7.79224634 7.06140487 4.29875823 3.15327191 3.11262912 2.84475108
 2.86087896 2.30480902 2.20761071 2.08383264 1.95510932 1.75904183
 1.61439339 1.4507434  1.47623068 1.46492695 1.34556852 1.33524915
 1.18799297 1.12838748 1.10666872 0.99358235 0.86449028 0.72416655
 0.8268461  0.73294778 0.62781363 0.73700962 0.6755728  0.69689772
 0.6870893  0.77070754 0.71583866 0.59031551 0.55729969 0.47250702
 0.56529618 0.5099707  0.42597663 0.47820164 0.47050986 0.41112936
 0.47032268 0.45100098 0.27196266 0.2818157  0.20497496]
Pr_G
[21 19 41  8 17  8  5 10 24 28 26 44  8 42 15 35  8 47  7 31  6 26 12 10
  5  7 21 40 23 26 15 12 15 46 18 30 33 45 14 23 26 10 13 12  6 34 46]
[ 0.8515591   0.76345272  2.96832718  0.36927697  0.68218805  0.36927697
  0.27085115  0.43428879  1.          1.23983023  1.11278747  4.23464426
  0.36927697  3.28711331  0.60648432  1.86840584  0.36927697 10.00454955
  0.33689009  1.46587148  0.30421829  1.11278747  0.50091222  0.43428879
  0.27085115  0.33689009  0.

/tmp/ipykernel_255069/3462460321.py:25: RuntimeWarning: divide by zero encountered in divide
  A_m = np.where(A_m!=0, 1/A_m,0) # Find a better way to compute it and avoid warning


In [13]:
# If priors are set as arrays
print('Pr_T')
Pr_T = norm.rvs(loc=T_m, scale=T_sd, size=K)
print(Pr_T)

print('Pr_G')
G_m, G_sd = 0, 1 # Don't know the actual values yet
s = 1 # Shape parameter for distribution, don't know actual useful value for this yet
N = K # Bins of equal probability, don't know if is K or a value sufficiently high
G_bins = randint.rvs(low=1, high=N+1, loc=0, size=K)
Pr_G = lognorm.ppf(q=(G_bins-0.5)/N, s=s, loc=G_m, scale=G_sd)
print(G_bins)
print(Pr_G)

print('Pr_A')
cluster_R = np.array(cluster_df['Rgc']).reshape(-1,1)[cluster_sort_idx] # Is Rgc the radius in pc? If not, where to find cluster radius
A_m = distance_matrix(cluster_R, cluster_R, p=1)
A_m = np.where(A_m!=0, 1/A_m,0) # Find a better way to compute it and avoid warning
print(A_m)
Pr_A = expon.rvs(loc=0, scale=A_m, size=(K,K))
print(Pr_A)

Pr_T
[8.06665767 6.89360987 4.22247503 3.14185119 3.14329456 2.8002966
 2.86369663 2.36552269 2.28879279 2.02281958 1.98218106 1.75071148
 1.66649197 1.47720418 1.51200365 1.46541465 1.38173058 1.34934655
 1.21360562 1.12067148 1.10635643 1.0143695  0.87302245 0.95943003
 0.69275574 0.80166826 0.8210735  0.77289202 0.8331078  0.57148991
 0.5115292  0.67016201 0.62952254 0.63241409 0.66636884 0.51159531
 0.50206336 0.5308325  0.63303038 0.46660062 0.49148558 0.50416503
 0.50510211 0.33831585 0.26101513 0.19390148 0.23401942]
Pr_G
[21 22 14 41 27 29  3  2 30 37 40 37  9 28 34 18  9 46 20 34  5  9 16 41
 17 23  8 14  1 21 41 13 14 29 39 12 29 34 34 28 28 12 21 42 43 11 25]
[0.8515591  0.89864419 0.5703624  2.96832718 1.17431661 1.30983881
 0.19895705 0.15670865 1.3849458  2.13987298 2.7079945  2.13987298
 0.40166916 1.23983023 1.75327127 0.72204992 0.40166916 6.38126853
 0.80656204 1.75327127 0.27085115 0.40166916 0.643714   2.96832718
 0.68218805 0.94804072 0.36927697 0.5703624  0.099954

/tmp/ipykernel_255069/412380374.py:18: RuntimeWarning: divide by zero encountered in divide
  A_m = np.where(A_m!=0, 1/A_m,0) # Find a better way to compute it and avoid warning


# Initialization

In [14]:
'''# If the priors were set as functions
T = T_m # Pr_T(loc=T_m, scale=T_sd,size=K)

G_m, G_sd = 0, 1 # Don't know the actual values yet
s = 1 # Shape parameter for distribution, don't know actual useful value for this yet
N = K # Bins of equal probability, don't know if is K or a value sufficiently high
G_bins = randint.rvs(low=1, high=N+1, loc=0, size=K)
G = Pr_G(s,G_m,G_sd).ppf(q=(G_bins-0.5)/N)

# TO ADD: Initialization for A''';

In [15]:
# If the priors were set as arrays
T = T_m # Pr_T
G = Pr_G

# TO ADD: Initialization for A
A = np.zeros((K+1,K))
'''(...)
Y = calY(T,G,X) 
Y_m, Y_sd = calY()
'''
A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

For the initialization of A:

- How to set Y[0] as the species abundance in pristine gas (75% H, 25% He)
- How to recover Y_m and Y_sd for all the clusters using calY (which I might need to implement)
- Look into the equation X[i]=Y_m[i]*b to obtain the off diagonal terms in A, check dimensions and implement solution of equations

In [16]:
# logLik and logPr
def logLik(T,G,X,Y_m,Y_sd,A):
    return

def logPr(T,T_m,T_sd,A,A_m):
    return